In [ ]:
import requests
import pandas as pd
from datetime import datetime

# URL da API Prometheus
url = 'http://192.168.242.131:9090/api/v1/query?query=container_cpu_usage_seconds_total{pod=~".*postgres.*"}[10m]'

# Fazendo a requisição GET
response = requests.get(url)
data = response.json()  # Converte a resposta para dict

# Verifica se há resultados e estrutura esperada
results = data.get('data', {}).get('result', [])

# Lista para armazenar os registros
records = []

# Processa cada resultado
for item in results:
    # Extrai a série temporal (samples)
    for value in item.get('values', []):
        ts = float(value[0])  # Timestamp UNIX
        cpu_val = float(value[1])
        # Converte timestamp para Data/Hora legível
        dt = datetime.utcfromtimestamp(ts)
        
        # Extrai o nome do pod dos labels
        pod_name = item['metric'].get('pod', 'N/A')
        records.append({"timestamp": dt, "pod": pod_name, "cpu": cpu_val})

# Cria DataFrame
df = pd.DataFrame(records)
#df = df.sort_values(by='timestamp', ascending=False)
# Mostra resultado
df

In [23]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import re

def query_prometheus_memory_with_instance(prometheus_url, pod_name, time_range="30m"):
    """
    Consulta memória do pod incluindo informações detalhadas de instance
    """
    print(f"\n🔍 Consultando memória do pod: {pod_name}")
    
    # Calcular timestamps Unix
    end_time = int(time.time())
    seconds = parse_time_range(time_range)
    start_time = end_time - seconds
    
    # Construir a query
    query = f'container_memory_usage_bytes{{pod="{pod_name}", container!="POD", container!=""}}'
    
    # Parâmetros com timestamps Unix
    params = {
        'query': query,
        'start': str(start_time),
        'end': str(end_time),
        'step': '30s'
    }
    
    api_url = f"{prometheus_url}/api/v1/query_range"
    
    print(f"🌐 URL da consulta: {api_url}")
    print(f"📋 Query: {query}")
    print(f"⏰ Start: {start_time} ({datetime.fromtimestamp(start_time)})")
    print(f"⏰ End: {end_time} ({datetime.fromtimestamp(end_time)})")
    print(f"📊 Período: {time_range}")
    
    try:
        print("📡 Fazendo requisição...")
        response = requests.get(api_url, params=params, timeout=30)
        
        print(f"📊 Status HTTP: {response.status_code}")
        
        if response.status_code != 200:
            print(f"❌ Erro HTTP {response.status_code}")
            print(f"Resposta: {response.text}")
            return pd.DataFrame()
        
        data = response.json()
        
        if data['status'] != 'success':
            print(f"❌ Erro na query Prometheus: {data.get('error', 'Erro desconhecido')}")
            print(f"Tipo do erro: {data.get('errorType', 'N/A')}")
            return pd.DataFrame()
        
        results = data['data']['result']
        
        if not results:
            print("⚠️  Nenhum dado encontrado para este pod")
            return pd.DataFrame()
        
        print(f"✅ Encontrados {len(results)} séries de dados")
        
        # Mostrar informações das instances encontradas
        instances_found = set()
        for result in results:
            metric = result['metric']
            instance = metric.get('instance', 'N/A')
            container = metric.get('container', 'N/A')
            instances_found.add(f"{instance} (container: {container})")
        
        print("\n🖥️  Instances encontradas:")
        for instance_info in sorted(instances_found):
            print(f"   📍 {instance_info}")
        
        # Processar dados
        processed_data = []
        
        for result in results:
            metric = result['metric']
            values = result['values']
            
            # Extrair todas as informações disponíveis da métrica
            pod_name_found = metric.get('pod', 'N/A')
            instance = metric.get('instance', 'N/A')
            container = metric.get('container', 'N/A')
            namespace = metric.get('namespace', 'N/A')
            node = metric.get('node', 'N/A')
            job = metric.get('job', 'N/A')
            
            # Separar IP e porta da instance (se disponível)
            instance_ip = 'N/A'
            instance_port = 'N/A'
            if instance and instance != 'N/A' and ':' in instance:
                try:
                    instance_ip, instance_port = instance.split(':')
                except:
                    instance_ip = instance
            elif instance and instance != 'N/A':
                instance_ip = instance
            
            print(f"📦 Processando - Container: {container}, Instance: {instance}")
            
            for timestamp, memory_bytes in values:
                try:
                    dt_utc = datetime.utcfromtimestamp(float(timestamp))
                    processed_data.append({
                        'pod_name': pod_name_found,
                        'instance': instance,
                        'instance_ip': instance_ip,
                        'instance_port': instance_port,
                        'container': container,
                        'namespace': namespace,
                        'node': node,
                        'job': job,
                        'datetime_utc': dt_utc.strftime('%Y-%m-%d %H:%M:%S'),
                        'timestamp': float(timestamp),
                        'memory_bytes': float(memory_bytes),
                        'memory_mb': round(float(memory_bytes) / (1024 * 1024), 2),
                        'memory_gb': round(float(memory_bytes) / (1024 * 1024 * 1024), 4)
                    })
                except (ValueError, TypeError) as e:
                    print(f"⚠️  Erro ao processar ponto de dados: {e}")
                    continue
        
        if not processed_data:
            print("❌ Nenhum dado válido foi processado")
            return pd.DataFrame()
        
        df = pd.DataFrame(processed_data)
        df = df.sort_values(['instance', 'container', 'timestamp']).reset_index(drop=True)
        
        print(f"✅ Dataset criado com {len(df)} registros")
        return df
        
    except Exception as e:
        print(f"❌ Erro inesperado: {e}")
        return pd.DataFrame()

def parse_time_range(time_range):
    """
    Converte time_range (ex: "30m", "1h", "2d") para segundos
    """
    match = re.match(r'(\d+)([smhd])', time_range.lower())
    if not match:
        raise ValueError(f"Formato de time_range inválido: {time_range}")
    
    value = int(match.group(1))
    unit = match.group(2)
    
    multipliers = {
        's': 1,      # segundos
        'm': 60,     # minutos
        'h': 3600,   # horas
        'd': 86400   # dias
    }
    
    return value * multipliers[unit]

def show_instance_summary(df):
    """
    Mostra resumo das instances encontradas
    """
    if df.empty:
        return
    
    print("\n" + "="*80)
    print("🖥️  RESUMO DAS INSTANCES")
    print("="*80)
    
    # Agrupar por instance
    instance_summary = df.groupby(['instance', 'container']).agg({
        'memory_mb': ['min', 'max', 'mean', 'count'],
        'pod_name': 'first',
        'namespace': 'first',
        'node': 'first'
    }).round(2)
    
    # Flatten column names
    instance_summary.columns = ['_'.join(col).strip() for col in instance_summary.columns.values]
    instance_summary = instance_summary.reset_index()
    
    for _, row in instance_summary.iterrows():
        print(f"\n📍 Instance: {row['instance']}")
        print(f"   �� Container: {row['container']}")
        print(f"   🏷️  Pod: {row['pod_name_first']}")
        print(f"   📂 Namespace: {row['namespace_first']}")
        print(f"   🖥️  Node: {row['node_first']}")
        print(f"   📊 Memória - Min: {row['memory_mb_min']} MB | Max: {row['memory_mb_max']} MB | Média: {row['memory_mb_mean']} MB")
        print(f"   📈 Registros: {int(row['memory_mb_count'])}")

def list_postgres_pods_with_instances(prometheus_url):
    """
    Lista pods PostgreSQL com suas respectivas instances
    """
    print("\n🔍 Buscando pods PostgreSQL e suas instances...")
    
    query = 'container_memory_usage_bytes{container!="POD", container!=""}'
    params = {'query': query}
    api_url = f"{prometheus_url}/api/v1/query"
    
    try:
        response = requests.get(api_url, params=params, timeout=15)
        
        if response.status_code != 200:
            print(f"❌ Erro ao listar pods: {response.status_code}")
            return []
        
        data = response.json()
        
        if data['status'] != 'success':
            print(f"❌ Erro na query: {data.get('error')}")
            return []
        
        # Extrair pods PostgreSQL com instances
        postgres_info = []
        all_pods_info = []
        
        for result in data['data']['result']:
            metric = result['metric']
            pod_name = metric.get('pod', '')
            instance = metric.get('instance', 'N/A')
            container = metric.get('container', 'N/A')
            namespace = metric.get('namespace', 'N/A')
            
            pod_info = {
                'pod': pod_name,
                'instance': instance,
                'container': container,
                'namespace': namespace
            }
            
            if pod_name:
                all_pods_info.append(pod_info)
                if 'postgres' in pod_name.lower():
                    postgres_info.append(pod_info)
        
        if postgres_info:
            print("✅ Pods PostgreSQL encontrados:")
            for info in postgres_info:
                print(f"   📦 Pod: {info['pod']}")
                print(f"      📍 Instance: {info['instance']}")
                print(f"      🔧 Container: {info['container']}")
                print(f"      📂 Namespace: {info['namespace']}")
                print()
        else:
            print("⚠️  Nenhum pod PostgreSQL encontrado")
            print("💡 Pods disponíveis (primeiros 5):")
            for info in all_pods_info[:5]:
                print(f"   📦 Pod: {info['pod']} | Instance: {info['instance']}")
        
        return [info['pod'] for info in postgres_info]
        
    except Exception as e:
        print(f"❌ Erro ao listar pods: {e}")
        return []

def main():
    # Configurações
    PROMETHEUS_URL = "http://192.168.242.131:9090"
    
    print("🚀 Iniciando consulta ao Prometheus com informações de Instance")
    print("=" * 80)
    
    # Listar pods PostgreSQL com instances
    postgres_pods = list_postgres_pods_with_instances(PROMETHEUS_URL)
    
    if postgres_pods:
        pod_name = postgres_pods[0]
        print(f"\n✅ Usando pod: {pod_name}")
    else:
        pod_name = input("\nDigite o nome exato do pod PostgreSQL: ").strip()
        if not pod_name:
            print("❌ Nome do pod não fornecido")
            return None
    
    # Executar consulta
    print("\n" + "=" * 80)
    df = query_prometheus_memory_with_instance(PROMETHEUS_URL, pod_name, "30m")
    
    if df.empty:
        print("❌ Nenhum dado foi retornado")
        return None
    
    # Mostrar resumo das instances
    show_instance_summary(df)
    
    # Exibir resultados detalhados
    print("\n" + "=" * 80)
    print("📊 DATASET COMPLETO")
    print("=" * 80)
    print(f"📈 Total de registros: {len(df)}")
    print(f"⏰ Período: {df['datetime_utc'].min()} até {df['datetime_utc'].max()}")
    print(f"🖥️  Instances únicas: {df['instance'].nunique()}")
    print(f"📦 Containers únicos: {df['container'].nunique()}")
    
    # Mostrar colunas disponíveis
    print(f"\n📋 Colunas do dataset:")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i:2d}. {col}")
    
    print(f"\n📋 Primeiros 5 registros:")
    # Mostrar apenas colunas principais para visualização
    display_cols = ['pod_name', 'instance', 'container', 'datetime_utc', 'memory_mb']
    print(df[display_cols].head().to_string(index=False))
    
    print(f"\n📊 Estatísticas Gerais de Memória:")
    print(f"   �� Mínima: {df['memory_mb'].min():.2f} MB")
    print(f"   🔺 Máxima: {df['memory_mb'].max():.2f} MB")
    print(f"   📊 Média: {df['memory_mb'].mean():.2f} MB")
    print(f"   📏 Desvio Padrão: {df['memory_mb'].std():.2f} MB")
    
    # Salvar arquivo com timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = f"postgres_memory_with_instances_{timestamp}.csv"
    df.to_csv(output_file, index=False)
    print(f"\n�� Dataset completo salvo em: {output_file}")
    
    # Salvar resumo por instance
    summary_file = f"postgres_instances_summary_{timestamp}.csv"
    instance_summary = df.groupby(['instance', 'container']).agg({
        'memory_mb': ['min', 'max', 'mean', 'std', 'count'],
        'pod_name': 'first',
        'namespace': 'first',
        'node': 'first'
    }).round(2)
    instance_summary.to_csv(summary_file)
    print(f"💾 Resumo por instance salvo em: {summary_file}")
    
    return df

if __name__ == "__main__":
    dataset = main()
    
    # Exemplo de como filtrar por instance específica
    if dataset is not None and not dataset.empty:
        print("\n" + "="*80)
        print("💡 EXEMPLO: Como filtrar por instance específica")
        print("="*80)
        
        instances = dataset['instance'].unique()
        if len(instances) > 0:
            example_instance = instances[0]
            filtered_df = dataset[dataset['instance'] == example_instance]
            print(f"Dados filtrados para instance '{example_instance}':")
            print(f"Total de registros: {len(filtered_df)}")
            print(filtered_df[['datetime_utc', 'container', 'memory_mb']].head(3).to_string(index=False))

🚀 Iniciando consulta ao Prometheus com informações de Instance

🔍 Buscando pods PostgreSQL e suas instances...
✅ Pods PostgreSQL encontrados:
   📦 Pod: postgres-deployment-76d6f4778d-ccs8g
      📍 Instance: x86
      🔧 Container: postgres
      📂 Namespace: stress-app


✅ Usando pod: postgres-deployment-76d6f4778d-ccs8g


🔍 Consultando memória do pod: postgres-deployment-76d6f4778d-ccs8g
🌐 URL da consulta: http://192.168.242.131:9090/api/v1/query_range
📋 Query: container_memory_usage_bytes{pod="postgres-deployment-76d6f4778d-ccs8g", container!="POD", container!=""}
⏰ Start: 1757245300 (2025-09-07 08:41:40)
⏰ End: 1757247100 (2025-09-07 09:11:40)
📊 Período: 30m
📡 Fazendo requisição...
📊 Status HTTP: 200
✅ Encontrados 1 séries de dados

🖥️  Instances encontradas:
   📍 x86 (container: postgres)
📦 Processando - Container: postgres, Instance: x86
✅ Dataset criado com 61 registros

🖥️  RESUMO DAS INSTANCES

📍 Instance: x86
   �� Container: postgres
   🏷️  Pod: postgres-deployment-76d6f4778d-

In [ ]:
df

,timestamp,pod,cpu
0,2025-09-07 11:35:13.859,postgres-deployment-76d6f4778d-ccs8g,32.653251
1,2025-09-07 11:35:33.830,postgres-deployment-76d6f4778d-ccs8g,32.658900
2,2025-09-07 11:35:46.481,postgres-deployment-76d6f4778d-ccs8g,32.661028
3,2025-09-07 11:36:00.848,postgres-deployment-76d6f4778d-ccs8g,32.663194
4,2025-09-07 11:36:17.275,postgres-deployment-76d6f4778d-ccs8g,32.665445
...,...,...,...
102,2025-09-07 11:43:37.589,postgres-deployment-76d6f4778d-ccs8g,0.054616
103,2025-09-07 11:43:52.151,postgres-deployment-76d6f4778d-ccs8g,0.054616
104,2025-09-07 11:44:12.135,postgres-deployment-76d6f4778d-ccs8g,0.054616
105,2025-09-07 11:44:23.583,postgres-deployment-76d6f4778d-ccs8g,0.054616


In [16]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 2.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import seaborn as sns

def query_prometheus_memory_for_chart(prometheus_url, pod_name, time_range="30m"):
    """
    Consulta dados do Prometheus para gerar gráfico
    """
    print(f"🔍 Consultando dados para gráfico - Pod: {pod_name}")
    
    # Calcular timestamps
    end_time = int(time.time())
    seconds = parse_time_range(time_range)
    start_time = end_time - seconds

  

    query = f'container_memory_usage_bytes{{pod=~".*{pod_name}.*", container!="POD", container!=""}}'
    
    params = {
        'query': query,
        'start': str(start_time),
        'end': str(end_time),
        'step': '30s'
    }
    
    api_url = f"{prometheus_url}/api/v1/query_range"
    
    try:
        response = requests.get(api_url, params=params, timeout=30)
        
        if response.status_code != 200:
            print(f"❌ Erro HTTP {response.status_code}")
            return pd.DataFrame()
        
        data = response.json()
        
        if data['status'] != 'success':
            print(f"❌ Erro Prometheus: {data.get('error')}")
            return pd.DataFrame()
        
        results = data['data']['result']
        
        if not results:
            print("⚠️  Nenhum dado encontrado")
            return pd.DataFrame()
        
        processed_data = []
        
        for result in results:
            metric = result['metric']
            values = result['values']
            
            pod_name_found = metric.get('pod', 'N/A')
            instance = metric.get('instance', 'N/A')
            container = metric.get('container', 'N/A')
            
            for timestamp, memory_bytes in values:
                try:
                    dt_utc = datetime.utcfromtimestamp(float(timestamp))
                    processed_data.append({
                        'pod_name': pod_name_found,
                        'instance': instance,
                        'container': container,
                        'datetime': dt_utc,
                        'timestamp': float(timestamp),
                        'memory_bytes': float(memory_bytes),
                        'memory_mb': round(float(memory_bytes) / (1024 * 1024), 2),
                        'memory_gb': round(float(memory_bytes) / (1024 * 1024 * 1024), 4)
                    })
                except (ValueError, TypeError):
                    continue
        
        if not processed_data:
            return pd.DataFrame()
        
        df = pd.DataFrame(processed_data)
        df = df.sort_values(['container', 'timestamp']).reset_index(drop=True)
        
        print(f"✅ {len(df)} registros coletados para {df['container'].nunique()} containers")
        return df
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return pd.DataFrame()

def parse_time_range(time_range):
    """Converte time_range para segundos"""
    match = re.match(r'(\d+)([smhd])', time_range.lower())
    if not match:
        raise ValueError(f"Formato inválido: {time_range}")
    
    value = int(match.group(1))
    unit = match.group(2)
    
    multipliers = {'s': 1, 'm': 60, 'h': 3600, 'd': 86400}
    return value * multipliers[unit]

def create_memory_time_chart(df, pod_name, save_file=True):
    """
    Cria gráfico de tempo vs memória
    """
    if df.empty:
        print("❌ Sem dados para gerar gráfico")
        return None
    
    # Configurar estilo
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    
    # Criar figura
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Agrupar por container
    containers = df['container'].unique()
    colors = plt.cm.Set3(range(len(containers)))
    
    for i, container in enumerate(containers):
        container_data = df[df['container'] == container].copy()
        container_data = container_data.sort_values('datetime')
        
        # Plotar linha
        ax.plot(container_data['datetime'], 
                container_data['memory_mb'],
                label=f'{container}',
                linewidth=2,
                marker='o',
                markersize=3,
                color=colors[i],
                alpha=0.8)
        
        # Adicionar área sombreada
        ax.fill_between(container_data['datetime'], 
                       container_data['memory_mb'],
                       alpha=0.2,
                       color=colors[i])
    
    # Configurar eixos
    ax.set_xlabel('Tempo (UTC)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Memória (MB)', fontsize=12, fontweight='bold')
    ax.set_title(f'Uso de Memória ao Longo do Tempo\nPod: {pod_name}', 
                fontsize=14, fontweight='bold', pad=20)
    
    # Formatação do eixo X (tempo)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=5))
    plt.xticks(rotation=45)
    
    # Formatação do eixo Y (memória)
    def format_mb(x, p):
        if x >= 1000:
            return f'{x/1000:.1f}GB'
        return f'{x:.0f}MB'
    
    ax.yaxis.set_major_formatter(FuncFormatter(format_mb))
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Legenda
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    
    # Estatísticas no gráfico
    stats_text = f"""Estatísticas Gerais:
• Mín: {df['memory_mb'].min():.1f} MB
• Máx: {df['memory_mb'].max():.1f} MB  
• Média: {df['memory_mb'].mean():.1f} MB
• Período: {df['datetime'].min().strftime('%H:%M')} - {df['datetime'].max().strftime('%H:%M')}"""
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', 
            facecolor='white', alpha=0.8), fontsize=9)
    
    # Ajustar layout
    plt.tight_layout()
    
    # Salvar arquivo
    if save_file:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'postgres_memory_chart_{timestamp}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"📊 Gráfico salvo como: {filename}")
    
    plt.show()
    return fig

def create_multiple_charts(df, pod_name):
    """
    Cria múltiplos tipos de gráficos
    """
    if df.empty:
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Gráfico de linha principal
    print("📊 Criando gráfico de linha...")
    create_memory_time_chart(df, pod_name, save_file=True)
    
    # 2. Gráfico por container (subplots)
    if df['container'].nunique() > 1:
        print("�� Criando gráficos separados por container...")
        
        containers = df['container'].unique()
        fig, axes = plt.subplots(len(containers), 1, figsize=(15, 4*len(containers)))
        
        if len(containers) == 1:
            axes = [axes]
        
        for i, container in enumerate(containers):
            container_data = df[df['container'] == container].copy()
            container_data = container_data.sort_values('datetime')
            
            axes[i].plot(container_data['datetime'], 
                        container_data['memory_mb'],
                        linewidth=2, marker='o', markersize=2)
            
            axes[i].fill_between(container_data['datetime'], 
                               container_data['memory_mb'], alpha=0.3)
            
            axes[i].set_title(f'Container: {container}', fontweight='bold')
            axes[i].set_ylabel('Memória (MB)')
            axes[i].grid(True, alpha=0.3)
            axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            
            # Estatísticas por container
            stats = f"Min: {container_data['memory_mb'].min():.1f}MB | Max: {container_data['memory_mb'].max():.1f}MB | Média: {container_data['memory_mb'].mean():.1f}MB"
            axes[i].text(0.02, 0.95, stats, transform=axes[i].transAxes, 
                        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
                        fontsize=8)
        
        plt.xlabel('Tempo (UTC)')
        plt.suptitle(f'Uso de Memória por Container - Pod: {pod_name}', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        filename = f'postgres_memory_by_container_{timestamp}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"📊 Gráfico por container salvo: {filename}")
        plt.show()
    
    # 3. Histograma de distribuição de memória
    print("📊 Criando histograma de distribuição...")
    
    plt.figure(figsize=(12, 6))
    
    containers = df['container'].unique()
    colors = plt.cm.Set3(range(len(containers)))
    
    for i, container in enumerate(containers):
        container_data = df[df['container'] == container]
        plt.hist(container_data['memory_mb'], bins=30, alpha=0.7, 
                label=container, color=colors[i], edgecolor='black')
    
    plt.xlabel('Memória (MB)', fontweight='bold')
    plt.ylabel('Frequência', fontweight='bold')
    plt.title(f'Distribuição do Uso de Memória - Pod: {pod_name}', 
              fontweight='bold', pad=20)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    filename = f'postgres_memory_distribution_{timestamp}.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"📊 Histograma salvo: {filename}")
    plt.show()

def main():
    # Configurações
    PROMETHEUS_URL = "http://192.168.242.131:9090"
    POD_NAME = "postgres"  # Ajuste conforme necessário
    TIME_RANGE = "130m"     # Período de consulta
    
    print("🚀 Gerando Gráficos de Memória vs Tempo")
    print("=" * 60)
    
    # Consultar dados
    df = query_prometheus_memory_for_chart(PROMETHEUS_URL, POD_NAME, TIME_RANGE)
    
    if df.empty:
        print("❌ Sem dados para gerar gráficos")
        return
    
    # Mostrar resumo dos dados
    print(f"\n📊 Resumo dos dados coletados:")
    print(f"   📈 Total de registros: {len(df)}")
    print(f"   ⏰ Período: {df['datetime'].min()} até {df['datetime'].max()}")
    print(f"   📦 Containers: {', '.join(df['container'].unique())}")
    print(f"   🖥️  Instances: {', '.join(df['instance'].unique())}")
    
    # Criar gráficos
    print(f"\n📊 Gerando gráficos...")
    create_multiple_charts(df, POD_NAME)
    
    print(f"\n✅ Gráficos gerados com sucesso!")
    
    return df

if __name__ == "__main__":
    # Instalar dependências necessárias:
    # pip install matplotlib seaborn requests pandas
    
    dataset = main()

🚀 Gerando Gráficos de Memória vs Tempo

📊 Resumo dos dados coletados:
   📈 Total de registros: 107


KeyError: 'datetime'